In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [2]:


from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter, PercentFormatter
import seaborn as sns
import matplotlib.lines as mlines


from utils import get_df

# Scalability plots


In [3]:
ALGO_LIST = [
    "mcpg_me",
    "dcg_me",
    "pga_me",
    "me",
    "memes",
    #"ppga",
]

In [4]:
results_dir = Path("fig2/output/")
EPISODE_LENGTH = 250
df = get_df(results_dir, EPISODE_LENGTH)
#df['algo'] = df.apply(filter, axis=1)
df = df[df["algo"].isin(ALGO_LIST)]
# df = df[df["batch_size"].isin(BATCH_LIST)]
df = df[df["num_evaluations"] <= 1_005_000]

ant_uni_250
walker2d_uni_250
ant_omni_250
hopper_uni_250
anttrap_omni_250


In [5]:
idx = df.groupby(["env", "algo", "run"])["iteration"].idxmax()
df_last_iteration = df.loc[idx]
scal_df = df_last_iteration[['env', 'algo', 'time', 'qd_score', 'batch_size']]

In [6]:
scal_df = scal_df.groupby(['env', 'algo', 'batch_size'])[['qd_score', 'time']].median().reset_index()
scal_df

,env,algo,batch_size,qd_score,time
0,ant_omni_250,dcg_me,256,966768.56250,2204.218018
1,ant_omni_250,dcg_me,512,967279.50000,1155.965088
2,ant_omni_250,dcg_me,1024,955885.93750,647.635864
3,ant_omni_250,dcg_me,2048,894473.15625,409.704010
4,ant_omni_250,dcg_me,4096,618278.06250,377.209717
...,...,...,...,...,...
140,walker2d_uni_250,pga_me,512,545788.25000,1190.351074
141,walker2d_uni_250,pga_me,1024,543705.71875,722.626465
142,walker2d_uni_250,pga_me,2048,538140.68750,636.280884
143,walker2d_uni_250,pga_me,4096,521937.62500,558.823608


In [7]:
grouped = scal_df.groupby(['env', 'algo'])['qd_score']
mean_qd_score = grouped.mean()
std_qd_score = grouped.std()

cv = std_qd_score / mean_qd_score * 100

cv_df = cv.reset_index(name='cv')
cv_df

,env,algo,cv
0,ant_omni_250,dcg_me,25.594073
1,ant_omni_250,mcpg_me,2.339835
2,ant_omni_250,me,4.829926
3,ant_omni_250,memes,67.027105
4,ant_omni_250,pga_me,10.871008
5,ant_uni_250,dcg_me,5.528535
6,ant_uni_250,mcpg_me,1.091570
7,ant_uni_250,me,1.035213
8,ant_uni_250,memes,32.503313
9,ant_uni_250,pga_me,10.582890


In [8]:
mean_cv_per_algo = cv_df.groupby('algo')['cv'].mean()
mean_cv_df = mean_cv_per_algo.reset_index(name='Mean_CV')
mean_cv_df

,algo,Mean_CV
0,dcg_me,13.436580
1,mcpg_me,2.440474
2,me,3.070694
3,memes,45.306304
4,pga_me,7.667352


In [11]:
pivot_df = df.pivot(index=['env', 'batch_size'], columns='algo', values='time')

In [12]:
pivot_df

algo                              dcg_me     mcpg_me
env              batch_size                         
ant_omni_250     256         2204.509766  468.571869
                 512         1157.795410  258.787170
                 1024         653.302856  164.696091
                 2048         409.809204  121.885239
                 4096         377.349976  108.978432
                 8192         325.654877   90.670830
ant_uni_250      256         2202.210938  466.288818
                 512         1155.803711  257.228516
                 1024         649.739868  163.386642
                 2048         414.132935  120.459183
                 4096         371.565674  107.580521
                 8192         325.380432   91.267433
anttrap_omni_250 256         2489.820312  752.919556
                 512         1314.591187  417.013306
                 1024         763.370850  276.249268
                 2048         472.838654  184.019501
                 4096         411.208862  142.981903
                 8192         410.878937  124.826797
hopper_uni_250   256         1856.447266  135.408142
                 512          961.403320   80.231941
                 1024         530.744751   62.195858
                 2048         325.980713   58.886993
                 4096         309.468323   63.197014
                 8192         269.542572   62.061096
walker2d_uni_250 256         1964.236084  222.667160
                 512         1032.785889  128.482666
                 1024         572.505493   90.432526
                 2048         371.066406   78.342896
                 4096         337.282837   77.458626
                 8192         289.841614   73.349426

In [13]:
pivot_df['ratio_256_to_8192'] = pivot_df['dcg_me'] / pivot_df['mcpg_me']

In [14]:
pivot_df

algo                              dcg_me     mcpg_me  ratio_256_to_8192
env              batch_size                                            
ant_omni_250     256         2204.509766  468.571869           4.704742
                 512         1157.795410  258.787170           4.473929
                 1024         653.302856  164.696091           3.966717
                 2048         409.809204  121.885239           3.362255
                 4096         377.349976  108.978432           3.462611
                 8192         325.654877   90.670830           3.591617
ant_uni_250      256         2202.210938  466.288818           4.722847
                 512         1155.803711  257.228516           4.493295
                 1024         649.739868  163.386642           3.976701
                 2048         414.132935  120.459183           3.437952
                 4096         371.565674  107.580521           3.453838
                 8192         325.380432   91.267433           3.565132
anttrap_omni_250 256         2489.820312  752.919556           3.306888
                 512         1314.591187  417.013306           3.152396
                 1024         763.370850  276.249268           2.763341
                 2048         472.838654  184.019501           2.569503
                 4096         411.208862  142.981903           2.875950
                 8192         410.878937  124.826797           3.291592
hopper_uni_250   256         1856.447266  135.408142          13.710012
                 512          961.403320   80.231941          11.982800
                 1024         530.744751   62.195858           8.533442
                 2048         325.980713   58.886993           5.535700
                 4096         309.468323   63.197014           4.896882
                 8192         269.542572   62.061096           4.343181
walker2d_uni_250 256         1964.236084  222.667160           8.821400
                 512         1032.785889  128.482666           8.038328
                 1024         572.505493   90.432526           6.330748
                 2048         371.066406   78.342896           4.736440
                 4096         337.282837   77.458626           4.354361
                 8192         289.841614   73.349426           3.951519

In [15]:
mean_ratio = pivot_df['ratio_256_to_8192'].mean()

In [16]:
mean_ratio

5.013537

In [9]:
BATCH_LIST = [
    256,
    8192,
]

In [10]:
df = df[df["batch_size"].isin(BATCH_LIST)]

In [11]:
df

,env,algo,batch_size,time
0,ant_omni_250,mcpg_me,256,468.571869
5,ant_omni_250,mcpg_me,8192,90.670830
6,ant_uni_250,mcpg_me,256,466.288818
11,ant_uni_250,mcpg_me,8192,91.267433
12,anttrap_omni_250,mcpg_me,256,752.919556
17,anttrap_omni_250,mcpg_me,8192,124.826797
18,hopper_uni_250,mcpg_me,256,135.408142
23,hopper_uni_250,mcpg_me,8192,62.061096
24,walker2d_uni_250,mcpg_me,256,222.667160
29,walker2d_uni_250,mcpg_me,8192,73.349426


In [12]:
pivot_df = df.pivot(index='env', columns='batch_size', values='time')

In [14]:
pivot_df['ratio_256_to_8192'] = pivot_df[256] / pivot_df[8192]

In [15]:
pivot_df

batch_size,256,8192,ratio_256_to_8192
env,,,
ant_omni_250,468.571869,90.670830,5.167835
ant_uni_250,466.288818,91.267433,5.109038
anttrap_omni_250,752.919556,124.826797,6.031714
hopper_uni_250,135.408142,62.061096,2.181852
walker2d_uni_250,222.667160,73.349426,3.035704


In [16]:
mean_ratio = pivot_df['ratio_256_to_8192'].mean()


In [17]:
mean_ratio

4.305228